In [1]:
import torch
import numpy as np

from env import make_pytorch_env
from decision_transformer.models.decision_transformer import DecisionTransformer


In [2]:
class MyClass:
    # vars to class
    def __init__(self, **kwargs):
        for key, value in kwargs.items():
            setattr(self, key, value)

In [3]:
loaded_model = torch.load("./exp/2023.03.16/170104-default/model.pt")
loaded_pretrain_model = torch.load("./exp/2023.03.16/170104-default/pretrain_model.pt")

variant = loaded_model['args']
args = MyClass(**variant)


In [4]:
def _get_env_spec(variant):
        #####env = gym.make(variant["env"])
        env = make_pytorch_env(args)
        state_dim = env.observation_space.shape[0]
        act_dim = env.action_space.shape[0]
        
        action_range = [
            float(env.action_space.low.min()) + 1e-6,
            float(env.action_space.high.max()) - 1e-6,
        ]
        
        env.close()
        return state_dim, act_dim, action_range

In [5]:
state_dim, act_dim, action_range = _get_env_spec(vars(args))
target_entropy = -act_dim

MAX_EPISODE_LEN = 4000

model = DecisionTransformer(
            state_dim=state_dim,
            act_dim=act_dim,
            action_range=action_range,
            max_length=variant["K"],
            eval_context_length=variant["eval_context_length"],
            max_ep_len=MAX_EPISODE_LEN,
            hidden_size=variant["embed_dim"],
            n_layer=variant["n_layer"],
            n_head=variant["n_head"],
            n_inner=4 * variant["embed_dim"],
            activation_function=variant["activation_function"],
            n_positions=1024,
            resid_pdrop=variant["dropout"],
            attn_pdrop=variant["dropout"],
            stochastic_policy=True,
            ordering=variant["ordering"],
            init_temperature=variant["init_temperature"],
            target_entropy=target_entropy,
        ).to(device=args.device)
        
        

/home/gabriel/miniconda3/envs/odt/lib/python3.8/site-packages/gym/spaces/box.py:84: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [89]:
#Load the weights on the model
model.load_state_dict(loaded_model['model_state_dict'])
# Set model to evaluation mode
model.eval()
# Convert model to GPU
model.to(device=args.device);

In [90]:
o = eval_fn(model)

AttributeError: 'Env' object has no attribute 'num_envs'

In [52]:
from evaluation import create_vec_eval_episodes_fn, vec_evaluate_episode_rtg
from stable_baselines3.common.vec_env import SubprocVecEnv

import pickle

In [53]:
def _load_dataset(env_name):

        dataset_path = f"./data/{env_name}.pkl"
        with open(dataset_path, "rb") as f:
            trajectories = pickle.load(f)

        states, traj_lens, returns = [], [], []
        for path in trajectories:
            states.append(path["observations"])
            traj_lens.append(len(path["observations"]))
            returns.append(path["rewards"].sum())
        traj_lens, returns = np.array(traj_lens), np.array(returns)

        # used for input normalization
        states = np.concatenate(states, axis=0)
        state_mean, state_std = np.mean(states, axis=0), np.std(states, axis=0) + 1e-6
        num_timesteps = sum(traj_lens)

        print("=" * 50)
        print(f"Starting new experiment: {env_name}")
        print(f"{len(traj_lens)} trajectories, {num_timesteps} timesteps found")
        print(f"Average return: {np.mean(returns):.2f}, std: {np.std(returns):.2f}")
        print(f"Max return: {np.max(returns):.2f}, min: {np.min(returns):.2f}")
        print(f"Average length: {np.mean(traj_lens):.2f}, std: {np.std(traj_lens):.2f}")
        print(f"Max length: {np.max(traj_lens):.2f}, min: {np.min(traj_lens):.2f}")
        print("=" * 50)

        sorted_inds = np.argsort(returns)  # lowest to highest
        num_trajectories = 1
        timesteps = traj_lens[sorted_inds[-1]]
        ind = len(trajectories) - 2
        while ind >= 0 and timesteps + traj_lens[sorted_inds[ind]] < num_timesteps:
            timesteps += traj_lens[sorted_inds[ind]]
            num_trajectories += 1
            ind -= 1
        sorted_inds = sorted_inds[-num_trajectories:]
        trajectories = [trajectories[ii] for ii in sorted_inds]

        return trajectories, state_mean, state_std

In [82]:
eval_envs = env

In [83]:
# Nao gostei disso pq tem a ver com o Dataset
offline_trajs, state_mean, state_std = _load_dataset(args.env)

Starting new experiment: drone_dataset
1544 trajectories, 3497627 timesteps found
Average return: -29.00, std: 2362.99
Max return: 3362.87, min: -5541.95
Average length: 2265.30, std: 1012.84
Max length: 4001.00, min: 919.00


In [84]:
reward_scale = 0.001

eval_fns = [
        create_vec_eval_episodes_fn(
            vec_env=eval_envs,
            eval_rtg=variant["eval_rtg"],
            state_dim=state_dim,
            act_dim=act_dim,
            state_mean=state_mean,
            state_std=state_std,
            device=args.device,
            use_mean=True,
            reward_scale=reward_scale,
        )
    ]


eval_fn = eval_fns[0]

In [85]:
online_envs = SubprocVecEnv(
                [
                    get_env_builder(i + 100, env_name=args.env, target_goal=None)
                    for i in range(variant["num_online_rollouts"])
                ]
            )

pybullet build time: May 20 2022 19:44:17


In [91]:
#Load the weights on the model
model.load_state_dict(loaded_model['model_state_dict'])
# Set model to evaluation mode
model.eval()
# Convert model to GPU
model.to(device=args.device)

    
# Load the Environment
env = make_pytorch_env(args)

/home/gabriel/miniconda3/envs/odt/lib/python3.8/site-packages/gym/spaces/box.py:84: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [95]:
#o = eval_fn(model)
o = eval_fn(model)

AttributeError: 'Env' object has no attribute 'num_envs'

array([ 3.89863277e+01,  3.93741047e+01,  1.39292683e+01, -1.28013207e-02])

In [64]:
state_pred, action_dist, reward_pred = model.get_predictions(
    (states.to(dtype=torch.float32) - state_mean) / state_std,
    actions.to(dtype=torch.float32),
    rewards.to(dtype=torch.float32),
    target_return.to(dtype=torch.float32),
    timesteps.to(dtype=torch.long),
    num_envs=num_envs,
)

NameError: name 'states' is not defined

In [26]:
model.get_predictions(torch_state)

TypeError: get_predictions() missing 4 required positional arguments: 'actions', 'rewards', 'returns_to_go', and 'timesteps'

In [24]:
torch_state =  torch.tensor(env.state).to(device=args.device)
model.predict_action(torch_state)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x4 and 512x3)

In [7]:
env.reset()
env.close()

terminal = False

while not terminal:
    
    
    with torch.no_grad():
        action = model(env.state)
    
    #action = np.array([3.0,3.0,3.0])
    next_state, reward, terminal, _ = env.step(action)
    env.render()


TypeError: forward() missing 5 required positional arguments: 'actions', 'rewards', 'returns_to_go', 'timesteps', and 'ordering'